In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
import tensorflow_text as text

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('spam.csv')

In [6]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [8]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
747/4825

0.15481865284974095

In [10]:
df_spam = df[df['Category'] == 'spam']
df_spam.shape

(747, 2)

In [11]:
df_ham = df[df['Category'] == 'ham']
df_ham.shape

(4825, 2)

In [12]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])

In [13]:
df_ham_downsampled.shape

(747, 2)

In [14]:
df_balanced = pd.concat([df_spam ,df_ham_downsampled ])
df_balanced.shape

(1494, 2)

In [15]:
df_balanced['Category'].value_counts()

Category
spam    747
ham     747
Name: count, dtype: int64

In [16]:
df_balanced['spam'] = df['Category'].apply(lambda x: 1 if x=='spam'  else 0 )

In [17]:
df_balanced.sample(5)

,Category,Message,spam
1897,ham,I tot u outside cos darren say u come shopping...,0
5482,spam,URGENT We are trying to contact you Last weeke...,1
837,spam,Do you want 750 anytime any network mins 150 t...,1
3709,spam,Free Top ringtone -sub to weekly ringtone-get ...,1
5504,ham,Wait . I will msg after &lt;#&gt; min.,0


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train , x_test , y_train , y_test = train_test_split(df_balanced['Message'] ,df_balanced['spam'] , stratify=df_balanced['spam'])

In [20]:
x_train.head(5)

3802              It's ok, at least armand's still around
1758                  Miss ya, need ya, want ya, love ya.
4692    Surly ill give it to you:-) while coming to re...
4179                   Two teams waiting for some players
5364    Call 09095350301 and send our girls into eroti...
Name: Message, dtype: object

In [21]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [25]:
def get_sentence_embedding(sentence):
    preprocessed_text = bert_preprocess(sentence)
    return bert_encoder(preprocessed_text)['pooled_output']
get_sentence_embedding(
    [
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.5132726 , -0.8884572 , ..., -0.74748856,
        -0.75314724,  0.91964495],
       [-0.8720835 , -0.50543964, -0.94446677, ..., -0.8584752 ,
        -0.7174535 ,  0.8808299 ]], dtype=float32)>

In [26]:
e = get_sentence_embedding(
    [
        "banana",
        "mango",
        "grapes",
        "jeff bezos",
        "elon musk",
        "SRK"
    ]
)

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
cosine_similarity([e[0]], [e[5]])

array([[0.9684411]], dtype=float32)